# Using glove with rnn

Using glove 6B token dataset
glove: https://nlp.stanford.edu/projects/glove/

Using gensim to load in glove, pre-converted into word2vec format for gensim

Download dataset from kaggle: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews
Extract dataset to dataset folder in the same directory of this notebook 

In [1]:
import gensim
import numpy as np
import tensorflow as tf
import pandas as pd
import string
import nltk
import re

/home/sw/Documents/dev/venvironment/mlenv/tf1_8env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# # convert glove to gensim word2vec format
# glove2word2vec('glove.6B.50d.txt', 'gloveWord2vec.txt')

In [3]:
df = pd.read_csv('./dataset/Womens Clothing E-Commerce Reviews.csv', index_col=0)

In [4]:
df

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [5]:
df2 = df.copy(deep=True)

In [6]:
df2.iloc[1, 3]

'Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.'

In [7]:
df2 = df2[['Review Text', 'Rating', 'Recommended IND']]

In [8]:
df2

,Review Text,Rating,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,4,1
1,Love this dress! it's sooo pretty. i happene...,5,1
2,I had such high hopes for this dress and reall...,3,0
3,"I love, love, love this jumpsuit. it's fun, fl...",5,1
4,This shirt is very flattering to all due to th...,5,1
5,"I love tracy reese dresses, but this one is no...",2,0
6,I aded this in my basket at hte last mintue to...,5,1
7,"I ordered this in carbon for store pick up, an...",4,1
8,I love this dress. i usually get an xs but it ...,5,1
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1


In [9]:
df2.dropna(how='any', inplace=True)

In [10]:
df2

,Review Text,Rating,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,4,1
1,Love this dress! it's sooo pretty. i happene...,5,1
2,I had such high hopes for this dress and reall...,3,0
3,"I love, love, love this jumpsuit. it's fun, fl...",5,1
4,This shirt is very flattering to all due to th...,5,1
5,"I love tracy reese dresses, but this one is no...",2,0
6,I aded this in my basket at hte last mintue to...,5,1
7,"I ordered this in carbon for store pick up, an...",4,1
8,I love this dress. i usually get an xs but it ...,5,1
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1


In [11]:
df3 = df2[['Review Text', 'Rating']]

In [12]:
df3

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5
5,"I love tracy reese dresses, but this one is no...",2
6,I aded this in my basket at hte last mintue to...,5
7,"I ordered this in carbon for store pick up, an...",4
8,I love this dress. i usually get an xs but it ...,5
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5


In [13]:
intToStringRatingDict = {1:'vbad', 2:'bad', 3:'meh', 4:'good', 5:'vgood'}

In [14]:
df3['Rating'] = df3['Rating'].apply(lambda x: intToStringRatingDict[x])

/home/sw/Documents/dev/venvironment/mlenv/tf1_8env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
df3

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,good
1,Love this dress! it's sooo pretty. i happene...,vgood
2,I had such high hopes for this dress and reall...,meh
3,"I love, love, love this jumpsuit. it's fun, fl...",vgood
4,This shirt is very flattering to all due to th...,vgood
5,"I love tracy reese dresses, but this one is no...",bad
6,I aded this in my basket at hte last mintue to...,vgood
7,"I ordered this in carbon for store pick up, an...",good
8,I love this dress. i usually get an xs but it ...,vgood
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",vgood


### preprocessing the sentences

In [16]:
# taking one as a sample sentence
sampleReview = df3.iloc[1, 0]

In [17]:
sampleReview

'Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.'

In [18]:
def preprocess(sentence):
    outputSentence = sentence.lower()
    outputSentence = replaceContractions(outputSentence)
    outputSentence = removePunc(outputSentence)
    outputSentence = removeNumbers(outputSentence)
    
    return outputSentence

In [19]:
def replaceContractions(sentence):
    outputSentence = sentence
    outputSentence = outputSentence.replace("won't", "will not")
    outputSentence = outputSentence.replace("can\'t", "can not")
    outputSentence = outputSentence.replace("n\'t", " not")
    outputSentence = outputSentence.replace("\'re", " are")
    outputSentence = outputSentence.replace("\'s", " is")
    outputSentence = outputSentence.replace("\'d", " would")
    outputSentence = outputSentence.replace("\'ll", " will")
    outputSentence = outputSentence.replace("\'t", " not")
    outputSentence = outputSentence.replace("\'ve", " have")
    outputSentence = outputSentence.replace("\'m", " am")
    return outputSentence

In [20]:
def removePunc(sentence):
    removePuncTrans = str.maketrans("", "", string.punctuation)
    outputSentence = sentence.translate(removePuncTrans)
    return outputSentence

In [21]:
def removeNumbers(sentence):
    outputSentence = sentence
    removeDigitsTrans = str.maketrans('', '', string.digits)
    outputSentence = outputSentence.translate(removeDigitsTrans)
    return outputSentence

In [22]:
replaceContractions(sampleReview)

'Love this dress!  it is sooo pretty.  i happened to find it in a store, and i am glad i did bc i never would have ordered it online bc it is petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.'

In [23]:
sampleReview

'Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.'

In [24]:
preprocess(sampleReview)

'love this dress  it is sooo pretty  i happened to find it in a store and i am glad i did bc i never would have ordered it online bc it is petite  i bought a petite and am   i love the length on me hits just a little below the knee  would definitely be a true midi on someone who is truly petite'

In [25]:
df3

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,good
1,Love this dress! it's sooo pretty. i happene...,vgood
2,I had such high hopes for this dress and reall...,meh
3,"I love, love, love this jumpsuit. it's fun, fl...",vgood
4,This shirt is very flattering to all due to th...,vgood
5,"I love tracy reese dresses, but this one is no...",bad
6,I aded this in my basket at hte last mintue to...,vgood
7,"I ordered this in carbon for store pick up, an...",good
8,I love this dress. i usually get an xs but it ...,vgood
9,"I'm 5""5' and 125 lbs. i ordered the s petite t...",vgood


In [26]:
df3['Review Text'] = df3['Review Text'].apply(preprocess)

/home/sw/Documents/dev/venvironment/mlenv/tf1_8env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
df3

,Review Text,Rating
0,absolutely wonderful silky and sexy and comfo...,good
1,love this dress it is sooo pretty i happened...,vgood
2,i had such high hopes for this dress and reall...,meh
3,i love love love this jumpsuit it is fun flirt...,vgood
4,this shirt is very flattering to all due to th...,vgood
5,i love tracy reese dresses but this one is not...,bad
6,i aded this in my basket at hte last mintue to...,vgood
7,i ordered this in carbon for store pick up and...,good
8,i love this dress i usually get an xs but it r...,vgood
9,i am and lbs i ordered the s petite to make ...,vgood


## word2vec preprocessing

Now to load the glove vectors into gensim as word2vec format, pre-converted earlier.

Also need a function to take in a sentence and return a stack of vectors of words in sentence, in order.

In [28]:
gloveModel = gensim.models.KeyedVectors.load_word2vec_format('gloveWord2vec.txt')

In [29]:
# try retrieving glove vector of word, using 50 dim version
gloveModel.get_vector('test')

array([ 0.13175 , -0.25517 , -0.067915,  0.26193 , -0.26155 ,  0.23569 ,
        0.13077 , -0.011801,  1.7659  ,  0.20781 ,  0.26198 , -0.16428 ,
       -0.84642 ,  0.020094,  0.070176,  0.39778 ,  0.15278 , -0.20213 ,
       -1.6184  , -0.54327 , -0.17856 ,  0.53894 ,  0.49868 , -0.10171 ,
        0.66265 , -1.7051  ,  0.057193, -0.32405 , -0.66835 ,  0.26654 ,
        2.842   ,  0.26844 , -0.59537 , -0.5004  ,  1.5199  ,  0.039641,
        1.6659  ,  0.99758 , -0.5597  , -0.70493 , -0.0309  , -0.28302 ,
       -0.13564 ,  0.6429  ,  0.41491 ,  1.2362  ,  0.76587 ,  0.97798 ,
        0.58507 , -0.30176 ], dtype=float32)

In [30]:
len(gloveModel.vocab)

400000

In [31]:
def getSentenceWordStack(sentence, gensimModel):
    sentenceWordList = sentence.split()
    vectorList = []
    for word in sentenceWordList:
        # to handle the case whereby the word in sentence is not in the vocab of the model.
        if word in gensimModel.vocab:
            vectorList.append(gensimModel.get_vector(word))
        else:
            vectorList.append(np.zeros(50)) # because we are using 50 dim vector, so 50 dim zero vector
    stackedVector = np.array(vectorList)
    return stackedVector

### Prepare the train, dev and test set

In [32]:
# create df4 which is a shuffed version of df3
df4 = df3.sample(frac=1).reset_index(drop=True)

In [33]:
df4.iloc[0, 0]

'very cute top but it seems like it will not hold it is shape very long cheaply constructed'

In [34]:
df3.iloc[0, 0]

'absolutely wonderful  silky and sexy and comfortable'

In [35]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22641 entries, 0 to 22640
Data columns (total 2 columns):
Review Text    22641 non-null object
Rating         22641 non-null object
dtypes: object(2)
memory usage: 353.8+ KB


In [36]:
traindf = df4.iloc[:17000, :]

In [37]:
devdf = df4.iloc[17000:20000, :]

In [38]:
testdf = df4.iloc[20000:, :]

In [39]:
traindf

,Review Text,Rating
0,very cute top but it seems like it will not ho...,good
1,this dress caught my eye because i do not have...,vgood
2,these jeans are my new favorites very flatteri...,vgood
3,i got an xs in this suit i am a this was m...,meh
4,i ordered the red color way and the bird print...,meh
5,i did not end up keeping this dressit just was...,meh
6,on the hanger this looks like a great lbd on m...,good
7,i love the look of this bra but unfortunately ...,good
8,a while back the moss ikat print appealed to m...,vgood
9,i got the ml and it is huge usually wear a lar...,meh


In [40]:
devdf

,Review Text,Rating
17000,yes it is boxy but by design because it is a s...,vgood
17001,i was not overly impressed or drawn to this dr...,good
17002,great sweater but it was just a little too sho...,good
17003,i ordered the xl in the green motif the shirt ...,good
17004,i really like the pilcro serif style but the s...,meh
17005,these pants fit the bill they are soft and ver...,vgood
17006,that is what i look like wearing this dress a ...,bad
17007,this was so unflattering the bust was too high...,bad
17008,i love these pants they are beautifully made a...,vgood
17009,oh such a classic retailer look for summer a v...,bad


In [41]:
testdf

,Review Text,Rating
20000,such a lovely shirt and even though it is whit...,vgood
20001,this wide leg pant gives the perception of a s...,vgood
20002,very cute summer dress it runs a bit large and...,good
20003,this vest is super cute i am and usually wea...,vgood
20004,for me this top ran true to size i am so glad ...,vgood
20005,when i got this skirt i figured it was a muted...,vgood
20006,like others i was drawn to the collar fabric a...,meh
20007,i like this sleeveless sweater it adds warmth...,vgood
20008,i love this shirt the neckline is flattering a...,vgood
20009,i love this dressbut and it saddened me that t...,vgood


When writing an RNN, it is possible to have arbitrary length input, because the recurrent units unfolds in time. However, when training, each batch needs to be of the same length. Here, I am going to try using a batchsize of 1, so that there is no need for padding, this is definitely not the best approach, just trying it out. It could bring instability to the training.

In [42]:
traindf['Review Text'] = traindf['Review Text'].apply(lambda x: getSentenceWordStack(x, gloveModel))

/home/sw/Documents/dev/venvironment/mlenv/tf1_8env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
# now we have converted all sentenced into stacks of word vectors
traindf

,Review Text,Rating
0,"[[0.57049, -0.0077854, -0.70766, -0.31785, 0.8...",good
1,"[[0.53074, 0.40117, -0.40785, 0.15444, 0.47782...",vgood
2,"[[1.0074, 0.18912, -0.11732, -0.36526, -0.0516...",vgood
3,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",meh
4,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",meh
5,"[[0.11890999972820282, 0.15254999697208405, -0...",meh
6,"[[0.30045, 0.25006, -0.16692, 0.1923, 0.026921...",good
7,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",good
8,"[[0.2170500010251999, 0.46514999866485596, -0....",vgood
9,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",meh


In [44]:
devdf['Review Text'] = devdf['Review Text'].apply(lambda x: getSentenceWordStack(x, gloveModel))

/home/sw/Documents/dev/venvironment/mlenv/tf1_8env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
testdf['Review Text'] = testdf['Review Text'].apply(lambda x: getSentenceWordStack(x, gloveModel))

/home/sw/Documents/dev/venvironment/mlenv/tf1_8env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
# building a dictionary to map label class into one hot vectors and another one for reverse
labelToOneHotIndex = {}
oneHotMaxToLabel = {}
possibleLabel = ['good', 'vgood', 'meh', 'vbad', 'bad']
for i in range(len(possibleLabel)):
    label = possibleLabel[i]
    oneHotMaxToLabel[i] = label
    labelToOneHotIndex[label] = i

In [47]:
def getOneHotVector(index):
    # here building according to 5 classes
    outputVector = np.zeros(5)
    outputVector[index] = outputVector[index] + 1
    return outputVector

In [48]:
labelToOneHotIndex

{'bad': 4, 'good': 0, 'meh': 2, 'vbad': 3, 'vgood': 1}

In [49]:
oneHotMaxToLabel

{0: 'good', 1: 'vgood', 2: 'meh', 3: 'vbad', 4: 'bad'}

### now a function that builds the model

In [50]:
tf.keras.backend.clear_session()

In [51]:
def buildModel(inputShape):
    
    x_input = tf.keras.layers.Input(inputShape)
    
    x = tf.keras.layers.GRU(units=20, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, name='GRU1')(x_input)
    
    x = tf.keras.layers.GRU(units=20, dropout=0.3, recurrent_dropout=0.3, return_sequences=False, name='GRU2')(x)
    
    x = tf.keras.layers.Dense(units=15, activation='relu', name='dense_1')(x)
    
    x = tf.keras.layers.Dense(units=5, activation='softmax', name='softmax_output')(x)
    
    model = tf.keras.Model(inputs=x_input, outputs=x, name='rnn_glove')
    
    return model

In [52]:
rnnModel = buildModel([None, 50]) # arbitrary length of 50 dim word vectors

In [53]:
adamClip = tf.keras.optimizers.Adam(clipnorm=1.0)

In [54]:
rnnModel.compile(optimizer=adamClip, loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
rnnModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 50)          0         
_________________________________________________________________
GRU1 (GRU)                   (None, None, 20)          4260      
_________________________________________________________________
GRU2 (GRU)                   (None, 20)                2460      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                315       
_________________________________________________________________
softmax_output (Dense)       (None, 5)                 80        
Total params: 7,115
Trainable params: 7,115
Non-trainable params: 0
_________________________________________________________________


In [56]:
traindf2 = traindf.copy(deep=True)

In [57]:
traindf2

,Review Text,Rating
0,"[[0.57049, -0.0077854, -0.70766, -0.31785, 0.8...",good
1,"[[0.53074, 0.40117, -0.40785, 0.15444, 0.47782...",vgood
2,"[[1.0074, 0.18912, -0.11732, -0.36526, -0.0516...",vgood
3,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",meh
4,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",meh
5,"[[0.11890999972820282, 0.15254999697208405, -0...",meh
6,"[[0.30045, 0.25006, -0.16692, 0.1923, 0.026921...",good
7,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",good
8,"[[0.2170500010251999, 0.46514999866485596, -0....",vgood
9,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",meh


In [58]:
traindf2['Rating'] = traindf2['Rating'].apply(lambda x: labelToOneHotIndex[x])

In [59]:
traindf2['Rating'] = traindf2['Rating'].apply(getOneHotVector)

In [60]:
traindf2

,Review Text,Rating
0,"[[0.57049, -0.0077854, -0.70766, -0.31785, 0.8...","[1.0, 0.0, 0.0, 0.0, 0.0]"
1,"[[0.53074, 0.40117, -0.40785, 0.15444, 0.47782...","[0.0, 1.0, 0.0, 0.0, 0.0]"
2,"[[1.0074, 0.18912, -0.11732, -0.36526, -0.0516...","[0.0, 1.0, 0.0, 0.0, 0.0]"
3,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...","[0.0, 0.0, 1.0, 0.0, 0.0]"
4,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...","[0.0, 0.0, 1.0, 0.0, 0.0]"
5,"[[0.11890999972820282, 0.15254999697208405, -0...","[0.0, 0.0, 1.0, 0.0, 0.0]"
6,"[[0.30045, 0.25006, -0.16692, 0.1923, 0.026921...","[1.0, 0.0, 0.0, 0.0, 0.0]"
7,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...","[1.0, 0.0, 0.0, 0.0, 0.0]"
8,"[[0.2170500010251999, 0.46514999866485596, -0....","[0.0, 1.0, 0.0, 0.0, 0.0]"
9,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...","[0.0, 0.0, 1.0, 0.0, 0.0]"


In [61]:
manualEpoch = 5

In [62]:
for j in range(manualEpoch):
    print('Epoch: ', j)
    traindf2 = traindf2.sample(frac=1).reset_index(drop=True)
    inputList = traindf2.iloc[:,0].tolist()
    labelList = traindf2.iloc[:,1].tolist()
    for i in range(len(inputList)):
        if i % 1000 == 0:
            print(i)
        inputMatrix = inputList[i].reshape(1, -1, 50)
        labelMatrix = labelList[i].reshape(1, -1)
        rnnModel.fit(x=inputMatrix, y=labelMatrix, batch_size=1, epochs=1, verbose=0)

Epoch:  0
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Epoch:  1
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Epoch:  2
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Epoch:  3
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Epoch:  4
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


In [63]:
tf.keras.models.save_model(rnnModel, 'rnnModel.h5')

## Loading the model back for evaluation

In [64]:
trainedModel = tf.keras.models.load_model('rnnModel.h5')

In [65]:
x_dev = devdf.iloc[:,0].values

In [66]:
type(x_dev)

numpy.ndarray

In [68]:
x_dev_list = list(x_dev)

In [69]:
y_dev = devdf.iloc[:,1].apply(lambda x: labelToOneHotIndex[x])

In [70]:
y_dev

17000    1
17001    0
17002    0
17003    0
17004    2
17005    1
17006    4
17007    4
17008    1
17009    4
17010    1
17011    0
17012    1
17013    0
17014    1
17015    1
17016    2
17017    1
17018    0
17019    1
17020    2
17021    1
17022    1
17023    1
17024    4
17025    1
17026    0
17027    1
17028    2
17029    0
        ..
19970    0
19971    0
19972    0
19973    1
19974    4
19975    2
19976    1
19977    1
19978    1
19979    0
19980    4
19981    3
19982    0
19983    1
19984    2
19985    1
19986    1
19987    1
19988    1
19989    1
19990    0
19991    4
19992    1
19993    1
19994    1
19995    0
19996    1
19997    0
19998    0
19999    0
Name: Rating, Length: 3000, dtype: int64

In [71]:
y_dev_list = list(y_dev.values)

In [72]:
devdf

,Review Text,Rating
17000,"[[-0.38796, 0.20422, 0.31733, -0.41985, 0.4740...",vgood
17001,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",good
17002,"[[-0.026567, 1.3357, -1.028, -0.3729, 0.52012,...",good
17003,"[[0.11890999972820282, 0.15254999697208405, -0...",good
17004,"[[0.11890999972820282, 0.15254999697208405, -0...",meh
17005,"[[1.0074, 0.18912, -0.11732, -0.36526, -0.0516...",vgood
17006,"[[0.88387, -0.14199, 0.13566, 0.098682, 0.5121...",bad
17007,"[[0.53074, 0.40117, -0.40785, 0.15444, 0.47782...",bad
17008,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",vgood
17009,"[[-0.07029200345277786, 1.607800006866455, 0.6...",bad


In [73]:
len(x_dev_list)

3000

In [74]:
x_dev[4].shape

(21, 50)

In [75]:
correctCount = 0
for i in range(len(x_dev_list)):
        inputMatrix = x_dev_list[i].reshape(1, -1, 50)
        correctLabelIndex = y_dev_list[i]
        result = np.argmax(trainedModel.predict(inputMatrix))
        if result == correctLabelIndex:
            correctCount = correctCount + 1
        if i % 1000 == 0:
            print(i)
print('percentage correct: ', float(correctCount)/len(x_dev_list))

0
1000
2000
percentage correct:  0.6146666666666667


In [76]:
testdf

,Review Text,Rating
20000,"[[0.6101199984550476, 0.33511999249458313, -0....",vgood
20001,"[[0.53074, 0.40117, -0.40785, 0.15444, 0.47782...",vgood
20002,"[[0.57049, -0.0077854, -0.70766, -0.31785, 0.8...",good
20003,"[[0.53074, 0.40117, -0.40785, 0.15444, 0.47782...",vgood
20004,"[[0.15272, 0.36181, -0.22168, 0.066051, 0.1302...",vgood
20005,"[[0.27062, -0.36596, 0.097193, -0.50708, 0.373...",vgood
20006,"[[0.36808, 0.20834, -0.22319, 0.046283, 0.2009...",meh
20007,"[[0.11890999972820282, 0.15254999697208405, -0...",vgood
20008,"[[0.11891, 0.15255, -0.082073, -0.74144, 0.759...",vgood
20009,"[[0.11890999972820282, 0.15254999697208405, -0...",vgood


In [77]:
y_test = testdf.iloc[:, 1].apply(lambda x: labelToOneHotIndex[x])

In [78]:
y_test.describe()

count    2641.000000
mean        1.177963
std         1.019945
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         4.000000
Name: Rating, dtype: float64

In [79]:
y_test[y_test == 1].count()

1495

In [80]:
y_test[y_test == 2].count()

318

In [81]:
y_test[y_test == 3].count()

92

In [82]:
y_test[y_test == 4].count()

176

In [83]:
y_test[y_test == 0].count()

560

### Note here that I have used a modified cluster sampling for test instances, I wanted to get same number of sample for each class/cluster, so that I can have a test set with same number for each class

In [94]:
clusterTestDf = testdf.groupby('Rating', group_keys=False).apply(lambda x: x.sample(min(len(x), 98)))

In [95]:
# reset index and shuffled
clusterTestDf = clusterTestDf.sample(frac=1).reset_index(drop=True)

In [96]:
x_test_cluster = clusterTestDf.iloc[:, 0].values

In [97]:
y_test_cluster = clusterTestDf.iloc[:, 1].apply(lambda x : labelToOneHotIndex[x])

In [98]:
y_test_cluster

0      2
1      4
2      2
3      2
4      4
5      2
6      0
7      3
8      0
9      2
10     2
11     4
12     4
13     3
14     1
15     3
16     3
17     0
18     1
19     1
20     3
21     3
22     2
23     4
24     1
25     3
26     4
27     2
28     3
29     2
      ..
454    0
455    3
456    4
457    2
458    3
459    0
460    4
461    2
462    2
463    2
464    1
465    4
466    1
467    1
468    2
469    1
470    4
471    3
472    3
473    3
474    0
475    4
476    4
477    1
478    0
479    0
480    2
481    0
482    2
483    4
Name: Rating, Length: 484, dtype: int64

In [99]:
correctCount = 0
for i in range(len(x_test_strat)):
        inputMatrix = x_test_strat[i].reshape(1, -1, 50)
        correctLabelIndex = y_test_strat[i]
        result = np.argmax(trainedModel.predict(inputMatrix))
        if result == correctLabelIndex:
            correctCount = correctCount + 1
        if i % 1000 == 0:
            print(i)
print('cluster percentage correct: ', float(correctCount)/len(x_test_cluster))

0
cluster percentage correct:  0.32231404958677684


In [90]:
x_test_list = testdf.iloc[:,0].tolist()

In [91]:
y_test_list = list(y_test)

In [92]:
correctCount = 0
for i in range(len(x_test_list)):
        inputMatrix = x_test_list[i].reshape(1, -1, 50)
        correctLabelIndex = y_test_list[i]
        result = np.argmax(trainedModel.predict(inputMatrix))
        if result == correctLabelIndex:
            correctCount = correctCount + 1
        if i % 1000 == 0:
            print(i)
print('percentage correct: ', float(correctCount)/len(x_test_list))

0
1000
2000
percentage correct:  0.6020446800454373
